# ResNet50 With Pandas Python UDFs

We were unsuccessful in runing the code below with our data that was extracted in the preprocess section

# Launch Spark

Three configuration items have to be added to the Spark configuration to enable Arrow as it is disabled by default. This can be done without modifying SparkLauncher now, but you can just modify that if you like.

```python
    # Apache Arrow Config
    conf.set('spark.yarn.appMasterEnv.ARROW_PRE_0_15_IPC_FORMAT', '1')
    conf.set('spark.executorEnv.ARROW_PRE_0_15_IPC_FORMAT', '1')
    conf.set('spark.sql.execution.arrow.enabled', 'true')
```

In [1]:
!pip install keras  --no-cache-dir 
!pip install tensorflow --no-cache-dir

In [1]:
import import_ipynb
from data603 import SparkLauncher
from data603 import HDFS
#import extra libraries
import io
from io import StringIO, BytesIO

# get a configuration object
conf = SparkLauncher.get_spark_conf()

# add a file to the configuration that will get copied to all the nodes on the cluster
conf.set('spark.yarn.dist.files', 'keras_data/resnet50_weights_tf_dim_ordering_tf_kernels.h5')

# Apache Arrow Config
conf.set('spark.yarn.appMasterEnv.ARROW_PRE_0_15_IPC_FORMAT', '1')
conf.set('spark.executorEnv.ARROW_PRE_0_15_IPC_FORMAT', '1')
conf.set('spark.sql.execution.arrow.enabled', 'true')

# launch the cluster using the configuration
spark = SparkLauncher.get_spark_session(pack_venv = False, conf = conf)
hdfs = HDFS.get_hdfs()

importing Jupyter notebook from /scratch/data603/has1/data603/SparkLauncher.ipynb
Creating Spark Configuration
importing Jupyter notebook from /scratch/data603/has1/data603/HDFS.ipynb
Creating Spark Configuration
Setting Environment Variables
Creating Spark Session: has1_data603_spark_session


# Read Dataframe

This must be done _BEFORE_ the UDF is defined because the UDF needs the schemas of the dataframes it will be using. I have a parquet file written out with the bounding boxes extracted of several bird types.

In [2]:
image_chips = spark.read.parquet("/user/has1/chips_image.parquet")
#image_chips = image_chips.drop('data') # remove the full-image data.

In [3]:
image_chips.count()

6156

In [4]:
len(image_chips.columns)

5

In [5]:
#Checking to see its there
image_chips.show(10)


+----------------+---------+---------------+--------------------+--------------------+
|         ImageID|LabelName|      LabelText|           chip_data|           hdfs_path|
+----------------+---------+---------------+--------------------+--------------------+
|025d25975e4275a2| /m/0c29q|        Leopard|[FF D8 FF E0 00 1...|/user/has1/write_...|
|025d25975e4275a2| /m/0cd4d|        Cheetah|[FF D8 FF E0 00 1...|/user/has1/write_...|
|025d25975e4275a2| /m/0449p|Jaguar (Animal)|[FF D8 FF E0 00 1...|/user/has1/write_...|
|03bacd7be83b721e| /m/096mb|           Lion|[FF D8 FF E0 00 1...|/user/has1/write_...|
|078bfcf1afb210ae| /m/096mb|           Lion|[FF D8 FF E0 00 1...|/user/has1/write_...|
|078bfcf1afb210ae| /m/096mb|           Lion|[FF D8 FF E0 00 1...|/user/has1/write_...|
|078bfcf1afb210ae| /m/096mb|           Lion|[FF D8 FF E0 00 1...|/user/has1/write_...|
|0c9f40ea3014c553| /m/096mb|           Lion|[FF D8 FF E0 00 1...|/user/has1/write_...|
|0e0e38e4ffb1b727| /m/0c29q|        Leopard

In [6]:
image_chips.printSchema()

root
 |-- ImageID: string (nullable = true)
 |-- LabelName: string (nullable = true)
 |-- LabelText: string (nullable = true)
 |-- chip_data: binary (nullable = true)
 |-- hdfs_path: string (nullable = true)



# Add In a Grouping Column
 The code below creates a column that will allow me to form 10 processing groups. The 10 in `n % 10` can be modified to change the number of groups based on your dataframe size. This dataframe contains 10k images to run predictions on.
Basically this section adds in a new column to the dataframe because we don't have something to group by.In order To control the amount of data being sent to workers, create a counter for each row using the pyspark `monotonically_increasing_id()` function. I then create a simple UDF that performs a modulo on that counter to form a column that will contain group numbers.

In [7]:
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import col, lit, udf
from pyspark.sql.types import IntegerType

def group_id(n):
    ret = n % 10
    return ret

udf_group_id = udf(group_id, IntegerType())

# create the counter 1 - # of rows
image_chips = image_chips.withColumn("n", monotonically_increasing_id())

# modulo the counter to get a repeating pattern of 0,1,2,3,4,5,6,7,8,9 for the group number
image_chips = image_chips.withColumn("grp", udf_group_id("n"))

# Create Ouput Column(s)

Because Arrow is a shared data structure, the new process cannot just add to the memory space of the originating process. To overcome this, we must create empty columns to be populated by the UDF. I do this using the `lit` function to populate columns with literal values of an empty string and a 0.0.

In [8]:
# create two empty columns for result of udf
image_chips = image_chips.withColumn('prediction_label', lit(""))
image_chips = image_chips.withColumn('prediction_confidence', lit(0.0))

# Create the Pandas UDF

I create the Pandas UDF using the `@pandas_udf` decorator. The schema of the dataframe is passed into the UDF decorator. 
The input to the function (called `pdf` here) is a Pandas dataframe. This is simply a Pandas dataframe that is pointing to a __portion__ of the Spark dataframe. It is created __without copying__ any data from the Spark memory to the Python process memory so it's fast!

The output of the function is the _same_ Pandas dataframe. The function simply modifies the dataframe as it wants. Doing this is actually modifying the Spark in-memory data structure! This is why new empty columns had to be pre-allocated in the spark dataframe so the UDF could put new data in an existing empty memory location.

The function below uses Keras to generate a ResNet50 prediction model.

In [9]:
import pandas as pd

from pyspark.sql.functions import pandas_udf, PandasUDFType

@pandas_udf(image_chips.schema, PandasUDFType.GROUPED_MAP)
def evaluate_chip(pdf):
    # pdf is a pandas dataframe
    import io
    import os
    from keras.applications.resnet50 import ResNet50
    from keras.applications.resnet50 import preprocess_input
    from keras.applications.resnet50 import decode_predictions
    from keras.preprocessing.image import load_img
    from keras.preprocessing.image import img_to_array

    # Load Model Data
    model = ResNet50(weights = f'{os.getcwd()}/resnet50_weights_tf_dim_ordering_tf_kernels.h5',
                 include_top = True)

    # Create arrays to hold prediction outputs.
    prediction_label = []
    prediction_confidence = []
    for chip_data in pdf['chip_data']:

        # Load the image
        #error have within this line
        img = load_img(io.RawIOBase(chip_data), target_size = (224,224))

        # Prepare Image
        image = img_to_array(img)
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        image = preprocess_input(image)

        # Run prediction
        yhat = model.predict(image)

        # Decode Predictions
        label = decode_predictions(yhat)
        label = label[0][0]  # Get first prediction (most confident)
        
        # Save prediction results in arrays
        prediction_label.append(label[1])
        prediction_confidence.append(label[2])

    # Assign result array data to the correct columns in the pandas dataframe
    pdf['prediction_label'] = prediction_label
    pdf['prediction_confidence'] = prediction_confidence
    
    return pdf

# Group the Image Dataframe, Apply Pandas UDF

Using the group column to separate the data into processing chunks, call `apply` on each chunk to apply the Pandas UDF.

In [10]:
image_chips = image_chips.groupby('grp').apply(evaluate_chip)

# View the Result!

Since Spark does lazy evaluation, this next line will take some time to process, but if we've done everything right, we shouldn't blow up the cluster memory limits.

In [11]:
image_chips.select(['prediction_label', 'prediction_confidence']).show(100)
#image_chips.select(['prediction_label', 'prediction_confidence'])

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 35064)
Traceback (most recent call last):
  File "/usr/lib64/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib64/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib64/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib64/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/scratch/data603_virtualenv/has1/lib64/python3.6/site-packages/pyspark/accumulators.py", line 267, in handle
    poll(authenticate_and_accum_updates)
  File "/scratch/data603_virtualenv/has1/lib64/python3.6/site-packages/pyspark/accumulators.py", line 239, in poll
    r, _, _ = select.select([self.rfile], [], [], 1)
ValueError: filedescriptor out of range in 

Py4JJavaError: An error occurred while calling o477.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: 
Aborting TaskSet 11.0 because task 24 (partition 49)
cannot run anywhere due to node and executor blacklist.
Most recent failure:
Lost task 24.0 in stage 11.0 (TID 61, worker5.hdp-internal, executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/cloudera/parcels/CDH-6.2.0-1.cdh6.2.0.p0.967373/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in main
    process()
  File "/opt/cloudera/parcels/CDH-6.2.0-1.cdh6.2.0.p0.967373/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 367, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/cloudera/parcels/CDH-6.2.0-1.cdh6.2.0.p0.967373/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 283, in dump_stream
    for series in iterator:
  File "<string>", line 1, in <lambda>
  File "/opt/cloudera/parc

# Analyze the Result!

At this point the `image_chips` dataframe is any other Spark dataframe, so process it accordingly.

In [13]:
ic_summary = image_chips.filter('prediction_confidence > 0.90')\
                        .groupby('prediction_label')\
                        .count()\
                        .filter("count > 100")\
                        .sort(col("count").desc())

In [14]:
ic_summary.show()

Py4JJavaError: An error occurred while calling o584.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: 
Aborting TaskSet 21.0 because task 107 (partition 107)
cannot run anywhere due to node and executor blacklist.
Most recent failure:
Lost task 102.1 in stage 21.0 (TID 503, worker8.hdp-internal, executor 4): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/cloudera/parcels/CDH-6.2.0-1.cdh6.2.0.p0.967373/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in main
    process()
  File "/opt/cloudera/parcels/CDH-6.2.0-1.cdh6.2.0.p0.967373/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 367, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/cloudera/parcels/CDH-6.2.0-1.cdh6.2.0.p0.967373/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 283, in dump_stream
    for series in iterator:
  File "<string>", line 1, in <lambda>
  File "/opt/cloudera/

In [ ]:
spark.stop()